In [1]:
from wdtools import *

In [2]:
setID = 'Set003'

In [3]:
r2_df = pd.read_csv(os.path.join(inpath + '\\output\\to_review\\unmatched_df_Set003_r2_N.csv'))

In [4]:
cor_r2_2 = update_unmatched_df_r2(r2_df, setID, ml='N', export=False)

In [5]:
cor_r2_2.trsqq.values

array(['05N02W27B', '33S07VE', '33S07VE', '07S11W22CD', '05N01W05',
       '03S10W', '33S07VE09', '06S11W35C', '07N02W08CC', '24S09E30CD'],
      dtype=object)

In [6]:
df = combine_corrected_unmatched(setID, ml='N')

In [7]:
df.shape

(50, 25)

In [8]:
rev_df = reindex_data(df)

In [9]:
rev_df.shape

(178, 29)

In [10]:
s = 'r2'
ml = 'N'

In [11]:
notes = pd.read_csv(os.path.join(inpath + '\\output\\to_review\\', f'unmatched_df_{setID}_{s}_{ml}_notes.csv'))

In [12]:
notes

,wetdet_delin_number,field,cor_type,from,to
0,WD2017-0267,trsqq,section number,17,08
1,WD2021-0192,trsqq,QQ,C0,CB
2,WD2018-0242,trsqq,section number,00,30
3,WD2018-0242,trsqq,QQ,00,CD
4,WD2018-0242,parcel_id,parcel id,201,2000


In [13]:
df = r2_df

In [14]:
df = df.copy()[df.wetdet_delin_number.isin(notes.wetdet_delin_number.unique())]

In [15]:
df

,wetdet_delin_number,trsqq,parcel_id,address_location_desc,city,county,site_name,site_desc,latitude,longitude,...,received_date,response_date,reissuance_response_date,project_id,site_id,record_ID,notes,recyear,IDyear,missinglot
0,WD2021-0192,06S11W35C,106p,1848 NE Sal La Sea Dr,Lincoln City,Lincoln,Logan Culvert Replacement Project,WD2021-0192 Replacement of existing culvert wi...,45.008654,-124.008015,...,2021-04-13,2022-02-17,NaN,81783.0,135782,2660,Partial,2021,2021,N
6,WD2017-0267,07N02W17CC,"800,900,1000","29120 Dike Rd, Rainier",Rainier,Columbia,Columbia Crossing 07N02W17CC,Retail development,46.096642,-122.964890,...,2017-06-23,2017-07-26,NaN,58816.0,105996,2162,NaN,2017,2017,N
7,WD2018-0242,24S09E00,201,Multiple,Crescent,Klamath,Crescent Waste Water Proj,Waste water treatment facility and infrastruct...,43.461260,-121.696353,...,2018-04-26,2018-07-23,NaN,NaN,141553,2392,NaN,2018,2018,N


In [16]:
for wdID in df.wetdet_delin_number.unique():
    sel = df.wetdet_delin_number == wdID
    fields = notes.loc[notes.wetdet_delin_number == wdID, 'field'].unique()
    for field in fields:
        sel2 = (notes.wetdet_delin_number==wdID) & (notes.field==field)
        k = notes[sel2].shape[0]
        if k > 1:
            df.loc[sel, field] = df[sel][field].apply(lambda x: pad_string(x))
            cor_types = notes.loc[sel2, 'cor_type'].values
            for cor_type in cor_types:
                sel3 = (notes.wetdet_delin_number==wdID) & (notes.field==field) & (notes.cor_type==cor_type)
                ind = trsqq_cor_dict[cor_type]
                df.loc[sel, field] = df[sel][field].apply(lambda x: replace_str_index(x,
                                                                                      index=ind,
                                                                                      replacement=notes.loc[sel3, 'to'].values[0]))
        else:
            df.loc[sel, field] = df[sel][field].apply(lambda x: x.replace(notes.loc[sel2, 'from'].values[0],
                                                                        notes.loc[sel2, 'to'].values[0]))

In [17]:
df

,wetdet_delin_number,trsqq,parcel_id,address_location_desc,city,county,site_name,site_desc,latitude,longitude,...,received_date,response_date,reissuance_response_date,project_id,site_id,record_ID,notes,recyear,IDyear,missinglot
0,WD2021-0192,06S11W35C,106p,1848 NE Sal La Sea Dr,Lincoln City,Lincoln,Logan Culvert Replacement Project,WD2021-0192 Replacement of existing culvert wi...,45.008654,-124.008015,...,2021-04-13,2022-02-17,NaN,81783.0,135782,2660,Partial,2021,2021,N
6,WD2017-0267,07N02W08CC,"800,900,1000","29120 Dike Rd, Rainier",Rainier,Columbia,Columbia Crossing 07N02W17CC,Retail development,46.096642,-122.964890,...,2017-06-23,2017-07-26,NaN,58816.0,105996,2162,NaN,2017,2017,N
7,WD2018-0242,24S09E30CD,2000,Multiple,Crescent,Klamath,Crescent Waste Water Proj,Waste water treatment facility and infrastruct...,43.461260,-121.696353,...,2018-04-26,2018-07-23,NaN,NaN,141553,2392,NaN,2018,2018,N


In [18]:
wdID = 'WD2021-0192'

In [19]:
sel = df.wetdet_delin_number == wdID
fields = notes.loc[notes.wetdet_delin_number == wdID, 'field'].unique()

In [21]:
field = fields[0]

In [22]:
sel2 = (notes.wetdet_delin_number==wdID) & (notes.field==field)
k = notes[sel2].shape[0]

In [23]:
k

1

In [24]:
df[sel][field]

0    06S11W35C
Name: trsqq, dtype: object

In [25]:
x = '06S11W35C'

In [27]:
notes.loc[sel2, 'from'].values[0]

'C0'

In [30]:
pad_string('24S09E00')

'24S09E0000'

In [29]:
pad_string(x).replace(notes.loc[sel2, 'from'].values[0], notes.loc[sel2, 'to'].values[0])

'06S11W35CB'